# The problem

For images of $150\times 150$, we have a covariance matrix $\Sigma = 22500 \times 22500$. This is computationally expensive, as the matrix has $506,250,000$ entries. Our matrix is sampled from a normal distribution $\sim \mathcal{N}(0,\sigma^2)$ for $\sigma = \frac{5.4}{2\sqrt{2\log(2)}}$. This means that, given the entries we pass into the distribution (distances) are strictly positive, we can calculate confidences $\alpha$ such that:
$$\alpha = P_{\mathcal{N}(0,\sigma^2)}(d\leq\tau|d\geq 0),$$
$$P_{\mathcal{N}(0,\sigma^2)}(d\leq\tau|d\geq 0) = \frac{P_{\mathcal{N}(0,\sigma^2)}(0\leq d\leq \tau)}{P_{\mathcal{N}(0,\sigma^2)}(d\geq 0)}$$
The denominator is $\frac{1}{2}$ from basic properties of gaussian distributions, leaving us with the problem:

$$\alpha = 2P_{\mathcal{N}(0,\sigma^2)}(0\leq d\leq\tau) = 2(P_{\mathcal{N}(0,\sigma^2)}(d\leq\tau)-P_{\mathcal{N}(0,\sigma^2)}(d\leq 0))$$
These can be written in terms of the inverse CDF of the standard normal distribution under $\tau \rightarrow \frac{\tau}{\sigma}.$ Noting that the second term on the RHS is also $\frac{1}{2}$, the cutoff distance $\tau$ for a confidence $\alpha$ can be written as ($\Phi^{-1}$ is the inverse normal CDF):

$$\tau = \Phi^{-1}(\frac{1}{2}\alpha + \frac{1}{2})$$

Using this we can find some cutoffs of pixel distance $d_{ij}$ using this $\tau$. As we are working with $L_2$ norms, these distances directly translate as nighbourhoods of $n = \text{round}(\tau)$ from the diagonal of the large $\Sigma$. When this is translatd to $(\text{RA},\delta)$ norm, it will have to be handled with a direct mask

One way to do this is to set pixels outside of thee neighbourhood of $n$ to zero, this would be a banded diagonal matrix. This does however keep the extremely large dimensionality of $\Sigma$. Instead, it is proposed that the matrix uses an $n\times n$ cutout of the matrix, taken from $(0,0)$ to $(n,n)$, forming a matrix of blocks along the diagonal. This form of sparse matrix is called block diagonal and we formulate it as below.

## Terminology of the Block Diagonal Matrix.
The notation used for the formulation is "pythonic", using $\%$ to mean modulo division, $//$ to mean floor division. It is also set that $r=22500\; \% \;n$. Setting this allows for the handling of edge cases, as is seen later. $\Sigma_n$ discusses the matrix slice of siz $n\times n$ as discussed above.

## Formulation of Block Diagonal Matrix
The matrix $\Sigma$ is reconstructed:

$$
\Sigma \approx \hat{\Sigma} = \begin{cases} \text{if r=0} & \text{diag}(\Sigma_n,...,\Sigma_n) \\ \text{otherwise} & \text{diag}(\Sigma_n,...,\Sigma_n,\Sigma_r)
\end{cases}

$$


The block diagonal has properties:
$$
\hat{\Sigma}^{-1} = \begin{cases} \text{if r=0} & \text{diag}(\Sigma_n^{-1},...,\Sigma_n^{-1}) \\ \text{otherwise} & \text{diag}(\Sigma_n^{-1},...,\Sigma_n^{-1},\Sigma_r^{-1})
\end{cases}
$$

$$
|\hat{\Sigma}| = \begin{cases} \text{if r=0} & \prod_{i=0}^{22500//2} |\Sigma_n| \\ \text{otherwise} & \left(\prod_{i=0}^{22500//2} |\Sigma_n|\right) + |\Sigma_r|
\end{cases}
$$

And therefore the log-determinant becomes:

$$
\log|\hat{\Sigma}| = \begin{cases} \text{if r=0} & \sum_{i=0}^{22500//2} \log |\Sigma_n| \\ \text{otherwise} & \left(\sum_{i=0}^{22500//2} \log|\Sigma_n|\right) + |\Sigma_r|
\end{cases}
$$


What matters most computationally in our case, as we consider the multivariate gaussian negative log likelihood, is the malahanobis term, or the term that has the matrix inversion:

$$
\frac{1}{2}(x-\mu)^T\Sigma^{-1} (x-\mu)
$$

With the above formulation, a simple algorithm to calculate it can be formed:

1. calculate $(x-\mu)$ as a vector.
2. Loop over i=0 to i= 22500//n:

    3. Find $z= (x-\mu)$[i*n:(i+1)*n].
    4. Compute $z^T \Sigma_n^{-1} z$.
    5. Add to Malahanobis distance and return to step 2.
    
6. If $r\neq 0$, $z_r = (x-\mu)[i*n:]$.
7. Add this to the malahanobis distance.

Thus, we now have a way to calculate each term in the negative log-likelihood.

## Importing and adding functions from other notebooks, previously used

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import matplotlib.colors as mcolors
from scipy.linalg import cholesky, solve_triangular

In [2]:
image_size = 50 # image size in pixels
sigma = 5.4 / (2 * np.sqrt(2 * np.log(2))) # FWHM = 5.4

cmap = plt.cm.viridis.copy()  # Copy the viridis colormap
# Create a colormap where 0 is forced to white
cmap.set_bad(color='white')  # For masked values
cmap.set_under(color='white')  # For explicitly set zero values

In [3]:
def find_correlation_matrix(image_size,sigma):
    """
    - Inputs:
        - image_size: data dimensionality
        - sigma: standard deviation of the target distribution.
    - Outputs:
        - Correlation matrix: the corrlation matrix as specified in the algorithm in the markdown cells above.
    """
    d = find_d(image_size)

    C= (1 / np.sqrt(2 * np.pi * sigma**2)) * np.exp(-d**2 / (2 * sigma**2)) # Just the N(0,sigma^2)

    # Set the diagonal elements to 1
    np.fill_diagonal(C, 1)

    return C

def find_d(image_size):
    """
    - Inputs:
        - image_size: data dimensionality
        - sigma: standard deviation of the target distribution.
    - Outputs:
        - Correlation matrix: the corrlation matrix as specified in the algorithm in the markdown cells above.
    """
    x, y = np.meshgrid(np.arange(image_size), np.arange(image_size), indexing="ij")
    pixel_coords = np.stack((x.ravel(), y.ravel()), axis=1)
    i, j = pixel_coords[:, 0], pixel_coords[:, 1] 

    di = i[:, None] - i[None, :] 
    dj = j[:, None] - j[None, :] 
    d = 1.8*np.sqrt(di**2 + dj**2)
    return d